In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [3]:
loader = PyPDFLoader('../data/Meta AI.pdf')

In [4]:
documents = loader.load()

In [5]:
documents

[Document(metadata={'source': '../data/Meta AI.pdf', 'page': 0}, page_content="DatahandlingmethodsofMeta\nService Meta\nObject Storage Haystack\nRelational Database MetaRDS\nNoSQLDatabase Cassandra\nMessaging Kafka\nBlockchain Libra\nHereareone-lineexplanationsforeachofMeta'sdatahandlingmethods\n1. ObjectStorage(Haystack):Ahighlyscalable, fault-tolerant storagesystemforlargefilesandobjects.2. . RelationalDatabase(MetaRDS):Amanagedrelational databaseserviceforstructureddata, basedonMySQLandPostgreSQL.3. . NoSQLDatabase(Cassandra):Adistributed, open-sourceNoSQLdatabaseforhandlinglargeamountsof unstructuredorsemi-structureddata.4. . Messaging(Kafka):Adistributedstreamingplatformforhigh-throughput, fault-tolerantdataprocessingandevent-drivenarchitecture.5. . Blockchain(Libra): Adecentralized, open-sourceblockchainplatformforsecure,transparent, andscalabledatamanagement.ThesedatahandlingmethodsenableMetatoefficientlymanageandprocessvast amountsofdataacrossitsservices."),
 Document(metadata=

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [8]:
splitter

In [9]:
chunk_documents = splitter.split_documents(documents)

In [10]:
chunk_documents

[Document(metadata={'source': '../data/Meta AI.pdf', 'page': 0}, page_content="DatahandlingmethodsofMeta\nService Meta\nObject Storage Haystack\nRelational Database MetaRDS\nNoSQLDatabase Cassandra\nMessaging Kafka\nBlockchain Libra\nHereareone-lineexplanationsforeachofMeta'sdatahandlingmethods\n1. ObjectStorage(Haystack):Ahighlyscalable, fault-tolerant storagesystemforlargefilesandobjects.2. . RelationalDatabase(MetaRDS):Amanagedrelational databaseserviceforstructureddata, basedonMySQLandPostgreSQL.3. . NoSQLDatabase(Cassandra):Adistributed, open-sourceNoSQLdatabaseforhandlinglargeamountsof unstructuredorsemi-structureddata.4. . Messaging(Kafka):Adistributedstreamingplatformforhigh-throughput, fault-tolerantdataprocessingandevent-drivenarchitecture.5. . Blockchain(Libra): Adecentralized, open-sourceblockchainplatformforsecure,transparent, andscalabledatamanagement.ThesedatahandlingmethodsenableMetatoefficientlymanageandprocessvast amountsofdataacrossitsservices."),
 Document(metadata=

In [11]:
from langchain.vectorstores import FAISS

In [14]:
from langchain_ollama import OllamaEmbeddings

In [23]:
embeddings = OllamaEmbeddings(
    model="llama3.2:1b",
)

In [25]:
db = FAISS.from_documents(chunk_documents, embedding=embeddings)

In [29]:
from langchain_ollama.llms import OllamaLLM

In [30]:
llm = OllamaLLM(model = 'llama3.2:1b')

In [31]:
from langchain_core.prompts import ChatPromptTemplate

In [41]:
prompt = ChatPromptTemplate.from_template("""
                                        Answer the user's question by only using the given context.
                                        <context>
                                          {context}
                                        </context>
                                          Question: {input}
""")

In [42]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [43]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [44]:
retriever = db.as_retriever()

In [45]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BD3DE9A930>, search_kwargs={})

In [46]:
from langchain.chains.retrieval import create_retrieval_chain

In [47]:
retriever_chain = create_retrieval_chain(retriever, document_chain)

In [51]:
response = retriever_chain.invoke({"input": "what is the LLM model provided my meta"})

In [52]:
response

{'input': 'what is the LLM model provided my meta',
 'context': [Document(metadata={'source': '../data/Meta AI.pdf', 'page': 6}, page_content='PerformanceComparison:'),
  Document(metadata={'source': '../data/Meta AI.pdf', 'page': 4}, page_content='OLLaMA:\nOllamastandsfor(Omni-LayerLearningLanguageAcquisitionModel), anovel approachtomachinelearningthat promisestoredefinehowweperceivelanguageacquisitionandnaturallanguageprocessing. At itscore, Ollamaisagroundbreakingplatformthat democratizesaccesstolargelanguagemodels(LLMs)byenablinguserstorunthemlocallyontheirmachines.Developedwithavisiontoempowerindividualsandorganizations, Ollamaprovidesa'),
  Document(metadata={'source': '../data/Meta AI.pdf', 'page': 5}, page_content='user-friendlyinterfaceandseamlessintegrationcapabilities, makingit easierthanevertoleveragethepowerof LLMsforvariousapplicationsandusecases.\nHerearesomeexamplemodelsthat canbedownloaded:'),
  Document(metadata={'source': '../data/Meta AI.pdf', 'page': 4}, page_conte

In [53]:
response['answer']

"You've asked about OLLaMA, a novel approach to machine learning that promises to redefine how we perceive language acquisition and natural language processing. Specifically, you want to know more about the OLLaMA model itself."